# url: https://qiita.com/nekoumei/items/648726e89d05cba6f432

In [1]:
import pandas as pd
from scipy import stats
import joblib

## データの読み込み

In [2]:
mail_df = pd.read_csv('http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv')

## (4) データの準備
### 女性向けメールが配信されたデータを削除したデータを作成

In [3]:
male_df = mail_df[mail_df.segment != 'Womens E-Mail'].copy() # 女性向けメールが配信されたデータを削除
male_df['treatment'] = male_df.segment.apply(lambda x: 1 if x == 'Mens E-Mail' else 0) #介入を表すtreatment変数を追加

## (5) 集計による比較
### group_byとsummairseを使って集計

In [4]:
male_df.groupby('treatment').agg( # データのグループ化
    conversion_rate=('conversion', 'mean'), # グループごとのconversionの平均
    spend_mean=('spend', 'mean'), # グループごとのspendの平均
    count=('treatment', 'count') # グループごとのデータ数
)

,conversion_rate,spend_mean,count
treatment,,,
0,0.005726,0.652789,21306
1,0.012531,1.422617,21307


## (6) t検定を行う

In [5]:
### (a)男性向けメールが配信されたグループの購買データを得る
mens_mail = male_df[male_df.treatment==1].spend.values

### (b)メールが配信されなかったグループの購買データを得る
no_mail = male_df[male_df.treatment==0].spend.values

### (a)(b)の平均の差に対して有意差検定を実行する
stats.ttest_ind(mens_mail, no_mail)

TtestResult(statistic=5.300090294465472, pvalue=1.163200872605869e-07, df=42611.0)

## (7) セレクションバイアスのあるデータの作成
### バイアスのあるデータの作成

In [6]:
sample_rules = (male_df.history > 300) | (male_df.recency < 6) | (male_df.channel=='Multichannel')
biased_df = pd.concat([
    male_df[(sample_rules) & (male_df.treatment == 0)].sample(frac=0.5, random_state=1),
    male_df[(sample_rules) & (male_df.treatment == 1)],
    male_df[(~sample_rules) & (male_df.treatment == 0)],
    male_df[(~sample_rules) & (male_df.treatment == 1)].sample(frac=0.5, random_state=1)
], axis=0, ignore_index=True)

## (8) セレクションバイアスのあるデータで平均を比較
### groupbyを使って集計(Biased)

In [7]:
biased_df.groupby('treatment').agg( # データのグループ化
    conversion_rate=('conversion', 'mean'), # グループごとのconversionの平均
    spend_mean=('spend', 'mean'), # グループごとのspendの平均
    count=('treatment', 'count') # グループごとのデータ数
)
# 乱数でbiased_dfのデータを作成しているので数値は書籍とは異なる。（結論は変わらない）

,conversion_rate,spend_mean,count
treatment,,,
0,0.004540,0.557954,14757
1,0.013572,1.541704,17168


## (9) scipy.statsのttest_indを使ってt検定を行う(Biased)

In [8]:
## (a)男性向けメールが配信されたグループの購買データを得る
mens_mail_biased = biased_df[biased_df.treatment == 1].spend.values

## (b)メールが配信されなかったグループの購買データを得る
no_mail_biased = biased_df[biased_df.treatment == 0].spend.values

## (a)(b)の平均の差に対して有意差検定を実行
stats.ttest_ind(mens_mail_biased, no_mail_biased)

TtestResult(statistic=5.595867225527976, pvalue=2.213198413365413e-08, df=31923.0)

In [9]:
#ch2で利用するmale_df, biased_dfを保存する
joblib.dump(male_df, 'male_df.joblib')
joblib.dump(biased_df, 'biased_df.joblib')

['biased_df.joblib']